In [4]:
%matplotlib inline
from __future__ import print_function
import pandas, os, numpy as np

In [14]:
titanic_path = '../external_data/titanic/'
if not os.path.isfile(titanic_path + 'train.csv'):
    print ("Please download the titanic dataset from Kaggle and save to " + titanic_path + "train")
titanic_csv = pandas.read_csv(titanic_path + 'train.csv')
print (titanic_csv.info())
titanic_csv.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [21]:
adjusted_titanic_csv = titanic_csv.copy()



average_age = np.round(np.nanmean(adjusted_titanic_csv['Age']))
print ("average_age:", average_age)
print ("average_survival", np.nanmean(adjusted_titanic_csv['Survived']))

adjusted_titanic_csv['Age'] = map(lambda x: average_age if np.isnan(x) else x, adjusted_titanic_csv['Age'])

adjusted_titanic_csv = adjusted_titanic_csv[adjusted_titanic_csv.columns[[2,4,5,6,7,9,11,1]]]
adjusted_titanic_csv['Sex'] = map(lambda x: 1 if x == 'male' else 0, adjusted_titanic_csv['Sex'])

embarkedConversion = {
    'S': 0,
    'C': 1,
    'Q': 2,
}
adjusted_titanic_csv['Embarked'] = map(lambda x: embarkedConversion[x] if type(x) == str else 3, adjusted_titanic_csv['Embarked'])
adjusted_titanic_csv.head()

average_age: 30.0
average_survival 0.383838383838


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,1,22.0,1,0,7.2500,0,0
1,1,0,38.0,1,0,71.2833,1,1
2,3,0,26.0,0,0,7.9250,0,1
3,1,0,35.0,1,0,53.1000,0,1
4,3,1,35.0,0,0,8.0500,0,0


In [22]:
import random
random.seed(42)
split = int(0.9*len(adjusted_titanic_csv))
X_cols = adjusted_titanic_csv.columns[:-1]
Y_cols = adjusted_titanic_csv.columns[-1:]
X_train = adjusted_titanic_csv[X_cols][:split]
Y_train = adjusted_titanic_csv[Y_cols][:split]
X_test = adjusted_titanic_csv[X_cols][split:]
Y_test = adjusted_titanic_csv[Y_cols][split:]

X_train.describe().columns,Y_train.describe().columns, X_train.shape, Y_train.shape

(Index([u'Pclass', u'Sex', u'Age', u'SibSp', u'Parch', u'Fare', u'Embarked'], dtype='object'),
 Index([u'Survived'], dtype='object'),
 (801, 7),
 (801, 1))

In [23]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((801L, 7L), (801L, 1L), (90L, 7L), (90L, 1L))

In [97]:
import keras
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Dense(1024,input_shape=X_train.shape[1:], activation='linear'))
model.add(LeakyReLU(alpha=.001))
model.add(Dropout(0.25))
model.add(Dense(1024, activation='linear'))
model.add(LeakyReLU(alpha=.001))
model.add(Dropout(0.2))
model.add(Dense(512, activation='linear'))
model.add(LeakyReLU(alpha=.001))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.summary(), model.optimizer.lr.get_value()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_23 (Dense)                 (None, 1024)          8192        dense_input_7[0][0]              
____________________________________________________________________________________________________
leakyrelu_17 (LeakyReLU)         (None, 1024)          0           dense_23[0][0]                   
____________________________________________________________________________________________________
dropout_17 (Dropout)             (None, 1024)          0           leakyrelu_17[0][0]               
____________________________________________________________________________________________________
dense_24 (Dense)                 (None, 1024)          1049600     dropout_17[0][0]                 
___________________________________________________________________________________________

(None, array(0.0010000000474974513, dtype=float32))

In [98]:
model.optimizer.lr.set_value(0.0001)

filepath="local_data/checkpoint.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [99]:
if os.path.isfile(filepath):
    try:
        model.load_weights(filepath)
        print ("checkpoint loaded")
    except:
        print ("invalid checkpoint")
    
model.fit(X_train, Y_train, nb_epoch=1000, batch_size=1024, validation_data=(X_test, Y_test), verbose=2, callbacks=callbacks_list)

checkpoint loaded
Train on 801 samples, validate on 90 samples
Epoch 1/1000
Epoch 00000: val_loss improved from inf to 0.32070, saving model to local_data/checkpoint.hdf5
0s - loss: 0.3769 - val_loss: 0.3207
Epoch 2/1000
Epoch 00001: val_loss improved from 0.32070 to 0.31979, saving model to local_data/checkpoint.hdf5
0s - loss: 0.3580 - val_loss: 0.3198
Epoch 3/1000
Epoch 00002: val_loss did not improve
0s - loss: 0.3505 - val_loss: 0.3204
Epoch 4/1000
Epoch 00003: val_loss did not improve
0s - loss: 0.3585 - val_loss: 0.3215
Epoch 5/1000
Epoch 00004: val_loss did not improve
0s - loss: 0.3536 - val_loss: 0.3225
Epoch 6/1000
Epoch 00005: val_loss did not improve
0s - loss: 0.3517 - val_loss: 0.3240
Epoch 7/1000
Epoch 00006: val_loss did not improve
0s - loss: 0.3568 - val_loss: 0.3251
Epoch 8/1000
Epoch 00007: val_loss did not improve
0s - loss: 0.3553 - val_loss: 0.3259
Epoch 9/1000
Epoch 00008: val_loss did not improve
0s - loss: 0.3510 - val_loss: 0.3267
Epoch 10/1000
Epoch 00009: 

In [100]:
titanic_csv_test = pandas.read_csv(titanic_path + 'test.csv')
titanic_csv_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [101]:
adjusted_titanic_csv_test = titanic_csv_test.copy()
        
adjusted_titanic_csv_test = adjusted_titanic_csv_test[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
adjusted_titanic_csv_test['Sex'] = map(lambda x: 1 if x=='male' else 0, adjusted_titanic_csv_test['Sex'])
adjusted_titanic_csv_test['Embarked'] = map(lambda x: embarkedConversion[x] if type(x)==str else 3, adjusted_titanic_csv_test['Embarked'])
adjusted_titanic_csv_test['Age'] = map(lambda x: average_age if np.isnan(x) else x, adjusted_titanic_csv_test['Age'])
adjusted_titanic_csv_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,2
1,3,0,47.0,1,0,7.0000,0
2,2,1,62.0,0,0,9.6875,2
3,3,1,27.0,0,0,8.6625,0
4,3,0,22.0,1,1,12.2875,0


In [102]:
X_input = np.array(adjusted_titanic_csv_test)
X_input.shape

(418L, 7L)

In [103]:
Y_output = model.predict(X_input)
Y_output.shape, Y_output[:20]

((418L, 2L), array([[  9.84567761e-01,   1.54321948e-02],
        [  9.28531706e-01,   7.14683309e-02],
        [  9.99999166e-01,   7.92655442e-07],
        [  8.27512681e-01,   1.72487259e-01],
        [  3.55775356e-01,   6.44224703e-01],
        [  6.05856061e-01,   3.94143969e-01],
        [  1.91442803e-01,   8.08557212e-01],
        [  9.94769394e-01,   5.23058791e-03],
        [  4.41974431e-01,   5.58025539e-01],
        [  9.99994516e-01,   5.44851991e-06],
        [  8.40884984e-01,   1.59115046e-01],
        [  5.27876616e-01,   4.72123355e-01],
        [  1.60300829e-07,   9.99999881e-01],
        [  9.90550399e-01,   9.44962166e-03],
        [  3.49402872e-05,   9.99965072e-01],
        [  4.27082693e-03,   9.95729148e-01],
        [  9.01836991e-01,   9.81630012e-02],
        [  2.17991814e-01,   7.82008171e-01],
        [  9.40313160e-01,   5.96867874e-02],
        [  4.29983497e-01,   5.70016503e-01]], dtype=float32))

In [104]:
survivers = map(lambda x: 0 if x[0]>0.25 else 1, Y_output)
survivers[:20], np.average(survivers)

([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0],
 0.29904306220095694)

In [105]:
import time
output = pandas.DataFrame(columns=['PassengerId','Survived'])
for i in xrange(len(survivers)):
    output.loc[i] = (892+i, survivers[i])
name = 'local_data/'+str(time.time())+'_dbuschho.csv'
output.to_csv(name,index=False)
print (name)
output[0:10]

local_data/1513462320.42_dbuschho.csv


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,1
7,899,0
8,900,0
9,901,0
